In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import plotly.graph_objects as go
from uncertainty_ellipsoid.config import PROCESSED_DATA_DIR
from uncertainty_ellipsoid.dataset import get_dataloader

2025-02-05 21:51:05.522 | INFO     | uncertainty_ellipsoid.config:<module>:11 - PROJ_ROOT path is: /Users/wsq/Codespace/2025/uncertainty_ellipsoid


In [5]:
# 创建数据加载器
dataloader = get_dataloader(
    h5_path=PROCESSED_DATA_DIR / "dataset.h5",
    batch_size=4,  
    shuffle=True,
    num_workers=0
)

In [9]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_batch_point_clouds(batch):
    """可视化一个batch的点云样本在4个子图中"""
    # 创建4个3D子图的布局
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
               [{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
        subplot_titles=[f'Sample {i+1}' for i in range(4)]
    )
    
    # 遍历batch中的前4个样本
    for i in range(4):
        world_coords = batch['world_coordinates'][i].numpy()  # (M_s, 3)
        pixel_coords = batch['pixel_coordinates'][i].numpy()  # (2,)
        depth = batch['depth'][i].item()                     # scalar
        
        # 添加点云到对应的子图
        fig.add_trace(go.Scatter3d(
            x=world_coords[:, 0],
            y=world_coords[:, 1],
            z=world_coords[:, 2],
            mode='markers',
            marker=dict(
                size=2,
                opacity=0.6
            ),
            name=f'Sample {i+1} (u={pixel_coords[0]:.1f}, v={pixel_coords[1]:.1f}, d={depth:.2f})'
        ), row=(i//2)+1, col=(i%2)+1)
    
    # 更新布局
    fig.update_layout(
        title='Batch Point Clouds with Uncertainty',
        showlegend=False
    )
    
    return fig

# 获取一个batch并可视化
batch = next(iter(dataloader))
fig = plot_batch_point_clouds(batch)
fig.show()
